In [ ]:
import os
import shutil
import re
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
from collections import Counter
import random
import pandas as pd
from datetime import datetime
from tqdm import tqdm


In [4]:
# invalid files in test
file_list = [
    '0gzwaq8-0.png',
    '0ilhsr2t-0.png',
    '1e1qxwyz-0.png',
    '1zcg2o-0.png',
    '2q9zjsun-0.png',
    '33yxl-0.png',
    '47v30-0.png',
    '4eyb0-0.png',
    '73z8hl-0.png',
    '7cbzihb3-0.png',
    '7ma7p-0.png',
    '8t5ylkn-0.png',
    '9kat-0.png',
    'ch9y67-0.png',
    'd9yfbv-0.png',
    'e59bi2u-0.png',
    'hy8d-0.png',
    'jbbjm4dp-0.png',
    'ka0c2tco-0.png',
    'kaq2m3-0.png',
    'm19yr21n-0.png',
    'med1omae-0.png',
    'n7xb-0.png',
    'n9t72-0.png',
    'ozv10-0.png',
    'q0t18y-0.png',
    'se9njc-0.png',
    'slhq5bp-0.png',
    't8nm319u-0.png',
    'yc4w-0.png',
    'z5bm2-0.png',
    'z63qv-0.png'
]


In [5]:
# get the valid files in the test
source_folder = './test'  
output_folder = './valid_test' 

# create the output folder if not exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Check the file name end with -0.png
def is_valid_filename(filename):
    # file name pattern
    pattern = r'^.+-0\.png$'
    return re.match(pattern, filename) is not None

for filename in os.listdir(source_folder):
    file_path = os.path.join(source_folder, filename)
    
    # check is there any sub folder
    if os.path.isfile(file_path):
        # check if the file is invalid file
        if filename not in file_list:
            if is_valid_filename(filename):
                dst_path = os.path.join(output_folder, filename)
            
                # move the file to output folder
                shutil.move(file_path, dst_path)
                print(f'Moved {filename} to {output_folder}')
            else:
                print(f'Skipped {filename} (invalid)')


Moved 002e23-0.png to ./valid_test
Moved 03yl9s-0.png to ./valid_test
Moved 03yuav5-0.png to ./valid_test
Moved 03zl9o-0.png to ./valid_test
Moved 04zqohgi-0.png to ./valid_test
Moved 05htm-0.png to ./valid_test
Moved 05pb-0.png to ./valid_test
Moved 07oj-0.png to ./valid_test
Moved 07z0-0.png to ./valid_test
Moved 08ft2e2z-0.png to ./valid_test
Moved 08nxd77-0.png to ./valid_test
Moved 08otejfi-0.png to ./valid_test
Moved 08sba-0.png to ./valid_test
Moved 09popm-0.png to ./valid_test
Moved 0a7sh2wp-0.png to ./valid_test
Moved 0al4pl-0.png to ./valid_test
Moved 0c3dp-0.png to ./valid_test
Moved 0c7mgdmd-0.png to ./valid_test
Moved 0chnm34-0.png to ./valid_test
Moved 0col7w-0.png to ./valid_test
Moved 0cr6-0.png to ./valid_test
Moved 0dx9-0.png to ./valid_test
Moved 0fa6qtkn-0.png to ./valid_test
Moved 0favtnx-0.png to ./valid_test
Moved 0fee-0.png to ./valid_test
Moved 0gil-0.png to ./valid_test
Moved 0h5ya8-0.png to ./valid_test
Moved 0i7uom-0.png to ./valid_test
Moved 0iqk95h-0.png t

In [6]:
# Data cleaning: black line removal
colors = set()

def fetch_random_image():
    return random.choice(os.listdir("valid_test"))

def get_pixel(img, i, j):
    if i < 0 or j < 0 or i >= img.shape[0] or j >= img.shape[1]:
        return None
    return img[i][j]

def read_image(image_name):
    return cv2.imread(f"valid_test/{image_name}")

def test_image(img):
    img = img.copy()
    max_colors = 10
    counter = Counter()
    for row in img:
        for pixel in row:
            counter[str(pixel)] += 1

    accepted_colours = [color for (color, _) in counter.most_common(max_colors + 2)]
    for i, row in enumerate(img):
        for j, pixel in enumerate(row):
            if str(pixel) == "[0 0 0]":
                # grab from surroundings:
                surrounding_pixels = [get_pixel(img, i + x, j + y) for x in range(-1, 2) for y in range(-1, 2) if x != 0 or y != 0 and str(pixel) in accepted_colours]
                fill = Counter(map(str, surrounding_pixels)).most_common(1)[0][0]
                fill_pixels = list(map(int, fill[1:-1].split()))
                # print(fill_pixels)
                for k in range(3):
                    img[i][j][k] = fill_pixels[k]
        

    return img


In [9]:
# Data cleaning: black line removal
colors = set()

def fetch_random_image():
    return random.choice(os.listdir("valid_test"))

def get_pixel(img, i, j):
    if i < 0 or j < 0 or i >= img.shape[0] or j >= img.shape[1]:
        return None
    return img[i][j]

def read_image(image_name):
    return cv2.imread(f"valid_test/{image_name}")

def test_image(img):
    img = img.copy()
    max_colors = 10
    counter = Counter()

    # Count all colors
    for row in img:
        for pixel in row:
            counter[str(pixel)] += 1

    accepted_colours = set(color for (color, _) in counter.most_common(max_colors + 2))

    for i, row in enumerate(img):
        for j, pixel in enumerate(row):
            if str(pixel) == "[0 0 0]":  # black pixel
                surrounding_pixels = []
                for x in range(-1, 2):
                    for y in range(-1, 2):
                        if x == 0 and y == 0:
                            continue
                        neighbor_pixel = get_pixel(img, i + x, j + y)
                        if neighbor_pixel is not None:
                            neighbor_str = str(neighbor_pixel)
                            if neighbor_str in accepted_colours:
                                surrounding_pixels.append(neighbor_str)
                
                if surrounding_pixels:
                    fill = Counter(surrounding_pixels).most_common(1)[0][0]
                    try:
                        fill_pixels = list(map(int, fill.strip("[]").split()))
                        img[i][j] = fill_pixels
                    except ValueError:
                        # fallback to white if conversion fails
                        img[i][j] = [255, 255, 255]
                else:
                    # No surrounding pixel to use; set white
                    img[i][j] = [255, 255, 255]

    return img


In [10]:
# remove the black lines and then put inside new folder
source_folder = './valid_test' 
output_folder = './clean_test'  

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for filename in os.listdir(source_folder):

    file_path = os.path.join(source_folder, filename)
    output_path = os.path.join(output_folder, filename)

    # read images
    img = cv2.imread(file_path)

    processed_image = test_image(img)

    # save images
    cv2.imwrite(output_path,processed_image)
    print(f'Saved {filename} to {output_folder}')

Saved 002e23-0.png to ./clean_test
Saved 03yl9s-0.png to ./clean_test
Saved 03yuav5-0.png to ./clean_test
Saved 03zl9o-0.png to ./clean_test
Saved 04zqohgi-0.png to ./clean_test
Saved 05htm-0.png to ./clean_test
Saved 05pb-0.png to ./clean_test
Saved 07oj-0.png to ./clean_test
Saved 07z0-0.png to ./clean_test
Saved 08ft2e2z-0.png to ./clean_test
Saved 08nxd77-0.png to ./clean_test
Saved 08otejfi-0.png to ./clean_test
Saved 08sba-0.png to ./clean_test
Saved 09popm-0.png to ./clean_test
Saved 0a7sh2wp-0.png to ./clean_test
Saved 0al4pl-0.png to ./clean_test
Saved 0c3dp-0.png to ./clean_test
Saved 0c7mgdmd-0.png to ./clean_test
Saved 0chnm34-0.png to ./clean_test
Saved 0col7w-0.png to ./clean_test
Saved 0cr6-0.png to ./clean_test
Saved 0dx9-0.png to ./clean_test
Saved 0fa6qtkn-0.png to ./clean_test
Saved 0favtnx-0.png to ./clean_test
Saved 0fee-0.png to ./clean_test
Saved 0gil-0.png to ./clean_test
Saved 0h5ya8-0.png to ./clean_test
Saved 0i7uom-0.png to ./clean_test
Saved 0iqk95h-0.png t

In [11]:
# do the segmantation by color
input_folder = './clean_test/'
output_folder = './segmat_test_filename/'
os.makedirs(output_folder, exist_ok=True) 


# define the background color: white
background_color = (255, 255, 255)

# extrict the colors from the image except the white background
def extract_colors(image, background_color):
    unique_colors = {}
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):
            pixel_color = tuple(image[y, x])
            if pixel_color != background_color:
                if pixel_color in unique_colors:
                    unique_colors[pixel_color] += 1
                else:
                    unique_colors[pixel_color] = 1
    sorted_colors = sorted(unique_colors.keys(), key=lambda x: unique_colors[x], reverse=True)
    return sorted_colors

# Sort colors from left to right
def sort_colors_by_position(image, colors):
    color_positions = []
    for color in colors:
        # find the pixels that having the same color
        y, x = np.where(np.all(image == color, axis=-1))
        # calculate the average x coordinate of the color (horizontal position)
        avg_x = np.mean(x)
        color_positions.append((avg_x, color))
    # sort by x coordinate
    color_positions.sort(key=lambda x: x[0])
    return [color for _, color in color_positions]

for file_name in os.listdir(input_folder):
    if file_name.endswith('.png'):
        input_filepath = os.path.join(input_folder, file_name)
        image = cv2.imread(input_filepath)
        
        if image is None:
            print(f'Note: can not read the file {input_filepath}')
            continue
        
        base_name = file_name.split('-')[0]  # extrict  the file name

        # Determine the number of the main colors based on the length of the name
        n_colors = len(base_name)  
        
        colors = extract_colors(image, background_color)
        # if more colors are deteced
        # then only keep the first n_colors based on the file nanme length
        if len(colors) > n_colors:
            colors = colors[:n_colors]
        
        sorted_colors = sort_colors_by_position(image, colors)
        # determine the file name order such as w72p -> ['w', '7', '2', 'p']
        graphic_names = list(base_name) 
        name_counter = {}
        
        plt.rcParams['figure.dpi'] = 300
        
        # some images have multiple same char
        # name them based on counter
        # such as '0d0vp1-0-1', '0d0vp1-0-2'
        for color, name in zip(sorted_colors, graphic_names):
            if name in name_counter:
                name_counter[name] += 1
            else:
                name_counter[name] = 1

            white_background = np.ones_like(image) * 255
            for y in range(image.shape[0]):
                for x in range(image.shape[1]):
                    # if the current pixel has the same color
                    if tuple(image[y, x]) == color:
                        white_background[y, x] = image[y, x]
            
            output_filename = os.path.join(output_folder, f'{base_name}-{name}-{name_counter[name]}.png')
            
            plt.figure(figsize=(image.shape[1] / 100, image.shape[0] / 100), dpi=300)
            plt.imshow(cv2.cvtColor(white_background, cv2.COLOR_BGR2RGB))
            plt.axis('off')
            plt.savefig(output_filename, bbox_inches='tight', pad_inches=0, dpi=300)
            plt.close()
            
            print(f'Saved {output_filename}')

Saved ./segmat_test_filename/002e23-0-1.png
Saved ./segmat_test_filename/002e23-0-2.png
Saved ./segmat_test_filename/002e23-2-1.png
Saved ./segmat_test_filename/002e23-e-1.png
Saved ./segmat_test_filename/002e23-2-2.png
Saved ./segmat_test_filename/002e23-3-1.png
Saved ./segmat_test_filename/03yl9s-0-1.png
Saved ./segmat_test_filename/03yl9s-3-1.png
Saved ./segmat_test_filename/03yl9s-y-1.png
Saved ./segmat_test_filename/03yl9s-l-1.png
Saved ./segmat_test_filename/03yl9s-9-1.png
Saved ./segmat_test_filename/03yl9s-s-1.png
Saved ./segmat_test_filename/03yuav5-0-1.png
Saved ./segmat_test_filename/03yuav5-3-1.png
Saved ./segmat_test_filename/03yuav5-y-1.png
Saved ./segmat_test_filename/03yuav5-u-1.png
Saved ./segmat_test_filename/03yuav5-a-1.png
Saved ./segmat_test_filename/03yuav5-v-1.png
Saved ./segmat_test_filename/03yuav5-5-1.png
Saved ./segmat_test_filename/03zl9o-0-1.png
Saved ./segmat_test_filename/03zl9o-3-1.png
Saved ./segmat_test_filename/03zl9o-z-1.png
Saved ./segmat_test_filen

In [ ]:
input_folder = './clean_test/'
output_folder = './segmat_test_frequency/'
less_folder = os.path.join(output_folder, 'less')
more_folder = os.path.join(output_folder, 'more')

os.makedirs(output_folder, exist_ok=True)
os.makedirs(less_folder, exist_ok=True)
os.makedirs(more_folder, exist_ok=True)

less_list = []
more_list = []
results = []
background_color = (255, 255, 255)  

def extract_colors_by_frequency(image, background_color):
    
    unique_colors = {}
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):
            pixel_color = tuple(image[y, x])
            if pixel_color != background_color:
                unique_colors[pixel_color] = unique_colors.get(pixel_color, 0) + 1
    
    sorted_colors = sorted(unique_colors.keys(), key=lambda x: unique_colors[x], reverse=True)
    sorted_frequencies = [unique_colors[color] for color in sorted_colors]
    return sorted_colors, sorted_frequencies

def find_frequency_cutoff(colors, frequencies, min_colors=4, max_colors=8):
    
    if not frequencies or len(frequencies) < min_colors:
        return colors[:min_colors] if len(colors) >= min_colors else colors
    
    cutoff_index = min_colors
    for i in range(min_colors, min(len(frequencies), max_colors)):
        if frequencies[i] < 0.5 * frequencies[i-1]:
            cutoff_index = i
            break
    else:
        cutoff_index = min(len(frequencies), max_colors)
    return colors[:cutoff_index]

def sort_colors_by_position(image, colors):
    
    color_positions = []
    for color in colors:
        y, x = np.where(np.all(image == color, axis=-1))
        avg_x = np.mean(x)
        color_positions.append((avg_x, color))
    color_positions.sort(key=lambda x: x[0])
    return [color for _, color in color_positions]

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_filename = os.path.join(output_folder, f'color_extraction_results_{timestamp}.csv')

for file_name in os.listdir(input_folder):
    if not file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue
        
    input_filepath = os.path.join(input_folder, file_name)
    image = cv2.imread(input_filepath)
    
    name_part = file_name.split('-')[0]
    name_length = len(name_part)
    
    colors, frequencies = extract_colors_by_frequency(image, background_color)
    selected_colors = find_frequency_cutoff(colors, frequencies)
    selected_count = len(selected_colors)
    
    if selected_count < name_length:
        save_folder = less_folder
        classification = 'less'
        less_list.append(file_name)
    elif selected_count > name_length:
        save_folder = more_folder
        classification = 'more'
        more_list.append(file_name)
    else:
        save_folder = output_folder
        classification = 'same'
    
    results.append({
        'filename': file_name,
        'name_part': name_part,
        'name_length': name_length,
        'sorted_colors': str(colors),
        'sorted_frequencies': str(frequencies),
        'selected_colors_count': selected_count,
        'classification': classification
    })
    
    sorted_colors = sort_colors_by_position(image, selected_colors)
    base_name = os.path.splitext(file_name)[0]
    graphic_names = list(name_part[:selected_count])
    
    name_counter = {}
    plt.rcParams['figure.dpi'] = 300
    
    for color, name in zip(sorted_colors, graphic_names):
        name_counter[name] = name_counter.get(name, 0) + 1
        
        white_background = np.ones_like(image) * 255
        white_background[np.all(image == color, axis=-1)] = image[np.all(image == color, axis=-1)]
        
        output_filename = os.path.join(save_folder, f'{base_name}-{name}-{name_counter[name]}.png')
        
        plt.figure(figsize=(image.shape[1]/100, image.shape[0]/100), dpi=300)
        plt.imshow(cv2.cvtColor(white_background, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        # plt.show()
        plt.savefig(output_filename, bbox_inches='tight', pad_inches=0, dpi=300)
        plt.close()

        print(f"Saved {output_filename}")


Saved ./segmat_test_frequency/002e23-0-0-1.png
Saved ./segmat_test_frequency/002e23-0-0-2.png
Saved ./segmat_test_frequency/002e23-0-2-1.png
Saved ./segmat_test_frequency/002e23-0-e-1.png
Saved ./segmat_test_frequency/002e23-0-2-2.png
Saved ./segmat_test_frequency/002e23-0-3-1.png
Saved ./segmat_test_frequency/less\03yl9s-0-0-1.png
Saved ./segmat_test_frequency/less\03yl9s-0-3-1.png
Saved ./segmat_test_frequency/less\03yl9s-0-y-1.png
Saved ./segmat_test_frequency/less\03yl9s-0-l-1.png
Saved ./segmat_test_frequency/less\03yl9s-0-9-1.png
Saved ./segmat_test_frequency/03yuav5-0-0-1.png
Saved ./segmat_test_frequency/03yuav5-0-3-1.png
Saved ./segmat_test_frequency/03yuav5-0-y-1.png
Saved ./segmat_test_frequency/03yuav5-0-u-1.png
Saved ./segmat_test_frequency/03yuav5-0-a-1.png
Saved ./segmat_test_frequency/03yuav5-0-v-1.png
Saved ./segmat_test_frequency/03yuav5-0-5-1.png
Saved ./segmat_test_frequency/03zl9o-0-0-1.png
Saved ./segmat_test_frequency/03zl9o-0-3-1.png
Saved ./segmat_test_frequenc

In [ ]:
df = pd.DataFrame(results)
df.to_csv(csv_filename, index=False)

print("charachter extracted less than actual length:", less_list)
print("charachter extracted more than actual length:", more_list)
print(f"\nsave result in: {csv_filename}")

with open(os.path.join(output_folder, 'less_list.txt'), 'w') as f:
    f.write("\n".join(less_list))
with open(os.path.join(output_folder, 'more_list.txt'), 'w') as f:
    f.write("\n".join(more_list))

颜色少于名字长度的文件: ['03yl9s-0.png', '0mgnhi-0.png', '0pv2w1t-0.png', '13bsgn2-0.png', '17z8030-0.png', '1dy0p-0.png', '1mrffz-0.png', '1ozltu-0.png', '21iyxsn-0.png', '2yjoie-0.png', '3j61hli-0.png', '3smh1i-0.png', '3ymdwwi-0.png', '5e8nthfj-0.png', '5mi1hb-0.png', '6046f2bl-0.png', '6885kiy-0.png', '6i4pw-0.png', '845ios-0.png', '95pcp5d-0.png', '9e1e5t3-0.png', '9ecqisn-0.png', '9ic2560-0.png', 'ahmw48r0-0.png', 'aiu0q3n5-0.png', 'akf8o1-0.png', 'api94-0.png', 'aybws-0.png', 'azci8sm-0.png', 'b45viav3-0.png', 'b842ks-0.png', 'c1dq8-0.png', 'cd2h9-0.png', 'difuege-0.png', 'eyq05-0.png', 'gczr1sen-0.png', 'gdqimd1p-0.png', 'god2llyl-0.png', 'gqbmo-0.png', 'h50769iw-0.png', 'h50uifd-0.png', 'i9dhn60-0.png', 'ibkmxh-0.png', 'il680q15-0.png', 'inrjbf6-0.png', 'iqawpl-0.png', 'j5nf05-0.png', 'knlb9c-0.png', 'kwwf5-0.png', 'ky0syh-0.png', 'm16cz-0.png', 'mbbip-0.png', 'mikvr626-0.png', 'mnn305bp-0.png', 'nqvdi-0.png', 'oxhpig0-0.png', 'qeltw3mx-0.png', 'spb6516e-0.png', 'stcnwr1u-0.png', 'uy0bk4

In [14]:
print(len(less_list))
print(len(more_list))

75
245


In [15]:
print(len(results))

1967


In [ ]:
def process_and_save_features(input_folder, output_folder, csv_path="test_dataset_info.csv"):

    features = []
    labels = []
    filenames = []
    error_log = []
    
    os.makedirs(output_folder, exist_ok=True)
    
    image_files = [f for f in os.listdir(input_folder) 
                  if f.lower().endswith(('.png'))]
    
    print(f"Starting process {len(image_files)} images ...")
    
    for filename in tqdm(image_files, desc="process"):
        try:
            parts = filename.split('-')
            if len(parts) < 3:
                raise ValueError("filename is wrong")
            
            label = parts[-2] 
            input_path = os.path.join(input_folder, filename)
            
            img = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                raise ValueError("cannot read image")
            
            _, binary = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            binary = 255 - binary 
            
            points = np.column_stack(np.where(binary < 127))
            if len(points) == 0:
                raise ValueError("no pixel")
            
            y_min, x_min = np.min(points, axis=0)
            y_max, x_max = np.max(points, axis=0)
            margin = 5
            char_img = binary[
                max(0, y_min-margin):min(binary.shape[0], y_max+margin),
                max(0, x_min-margin):min(binary.shape[1], x_max+margin)
            ]
            
            resized = cv2.resize(char_img, (32, 32), interpolation=cv2.INTER_AREA)
            
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, resized)
            
            features.append(resized.flatten())
            labels.append(label)
            filenames.append(filename)
            
        except Exception as e:
            error_log.append({
                "filename": filename,
                "error": str(e)
            })
    
    print("\nstore the results...")
    
    df = pd.DataFrame({
        "filename": filenames,
        "label": labels,
        "feature_shape": "32x32"
    })
    df.to_csv(csv_path, index=False)
    
    np.save(os.path.join(output_folder, "X_features.npy"), np.array(features))
    np.save(os.path.join(output_folder, "y_labels.npy"), np.array(labels))
    
    if error_log:
        error_df = pd.DataFrame(error_log)
        error_df.to_csv(os.path.join(output_folder, "processing_errors.csv"), index=False)
    
    print(f"\nFinishing:")
    print(f"success: {len(features)} ")
    print(f"fail: {len(error_log)} ")
    print(f"features stored in: {os.path.join(output_folder, 'X_features.npy')}")
    print(f"y stored in: {os.path.join(output_folder, 'y_labels.npy')}")
    print(f"row data: {csv_path}")
    if error_log:
        print(f"error log: {os.path.join(output_folder, 'processing_errors.csv')}")

input_folder = "./segmat_test_filename"
output_folder = "./test_dataset"
csv_path = "test_dataset_info.csv"

process_and_save_features(input_folder, output_folder, csv_path)

Starting process 11808 images ...


process: 100%|██████████| 11808/11808 [01:42<00:00, 114.96it/s]


store the results...

Finishing:
success: 11808 
fail: 0 
features stored in: ./test_dataset\X_features.npy
y stored in: ./test_dataset\y_labels.npy
row data: test_dataset_info.csv
